In [1]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow
import tqdm
from keras.callbacks import TensorBoard
from tensorflow.keras.metrics import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.data.Dataset as dt

In [ ]:
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

In [ ]:
DATASET_NAME = "emptyFitdetectionFit"

DATASET_FOLDER = os.path.abspath("./data/processed")
DATASET_PATH = os.path.join(DATASET_FOLDER, DATASET_NAME)

IMAGES_CSV = os.path.abspath(
    "data/interim/emptyNonEmptyDataset/28570Images_0_003_threshold.csv"
)

THRESHOLD_NAME = os.path.basename(IMAGES_CSV).split("Images_")[-1].replace(".csv", "")

FIT_PATH = os.path.abspath(f"./resources/fit_images/{THRESHOLD_NAME}")
ORIGINAL_PATH = os.path.abspath("./data/interim/emptyNonEmptyDataset")

POSITIVE_DETECTION = FIT_PATH
NEGATIVE_DETECTION = FIT_PATH

print(f"DATASET_PATH:       {DATASET_PATH}")
print()
print(f"IMAGES_CSV:         {IMAGES_CSV}")
print()
print(f"POSITIVE_DETECTION: {POSITIVE_DETECTION}")
print(f"NEGATIVE_DETECTION: {NEGATIVE_DETECTION}")

In [ ]:
images = dt.load_from_csv(IMAGES_CSV)

In [ ]:
for index in tqdm.tqdm(images.index):
    file_name = images.loc[index, "file_name"]
    megadetector_label = images.loc[index, "detector_label"]

    images.loc[index, "file_name_new"] = (
        os.path.join(POSITIVE_DETECTION, file_name)
        if megadetector_label == 1
        else os.path.join(NEGATIVE_DETECTION, file_name)
    )

images["binary_label"] = images["binary_label"].astype("str")
images

In [ ]:
for _, row in tqdm.tqdm(images.iterrows()):
    original_file = row["file_name_new"]
    new_dataset = os.path.join(DATASET_PATH, row["file_name"])

    os.makedirs(os.path.dirname(new_dataset), exist_ok=True)

    try:
        shutil.copyfile(original_file, new_dataset)
    except FileNotFoundError:
        print(f"File not found: {original_file}")

shutil.copyfile(IMAGES_CSV, os.path.join(DATASET_PATH, os.path.basename(IMAGES_CSV)))